# 문장학습(GSCV를 이용한 SVM)

In [ ]:
from konlpy.tag import *
kkma=Kkma()
tw=Twitter()

In [ ]:
def ngram(command):
    temp=command+' '
    for i in range(1,4):
        for j in range(len(command)-i+1):
            temp=temp+command[j:j+i]+' '
    return temp

class textList:
    def __init__(self):
        self.command=[]
        self.answer=[]
    def append(self,text,text2):
        self.command.append(text)
        self.answer.append(int(text2))
        
    def nouns(self):
        nouns_list=[]
        for nowC, nowA in zip(self.command, self.answer):
            tmp=''
            arr=tw.pos(nowC)
            arr2=kkma.pos(nowC)
            for pos, pos2 in zip(arr, arr2):
                if pos[1] == 'Noun' or pos[1] == 'Verb':
                    tmp=tmp+ngram(pos[0]) + ' '
                if pos2[1][0] =='N' or pos[1][0] == 'V':
                    tmp=tmp+ngram(pos2[0])+' '
            nouns_list.append((tmp, nowA))
        return nouns_list

```
1. 좌회전
2. 우회전
3. 좌걷기
4. 우걷기
5. 앞걷기
6. 뒤걷기
7. 공격
```

In [ ]:
motionDict={1:'좌회전',2:'우회전',3:'좌걷기',4:'우걷기',5:'앞걷기',6:'뒤걷기',7:'공격'}

In [ ]:
import csv
train = textList()
f = open('command.csv', 'r', encoding='utf-8')
rdr = csv.reader(f)
next(rdr)
for line in rdr:
    train.append(line[0], line[1])
f.close()

In [ ]:
f = open("x_text_list.out", 'w')
for now in train.nouns():
    f.write(now[0])
    f.write('\n')
f.close()

In [ ]:
from sklearn.model_selection import train_test_split
import re
import sys
from sklearn.externals import  joblib
from sklearn.grid_search import GridSearchCV
from sklearn.svm import  LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import  TfidfVectorizer,CountVectorizer
import os
import numpy as np
import string
import json
import nltk
vectorizer = CountVectorizer()

In [ ]:
svc_param = np.logspace(-2,0,1000)
gsvc = GridSearchCV(LinearSVC(), param_grid= {'C': svc_param}, cv = 2, n_jobs = 4)
x_text_list = list(map(lambda i : i[0],train.nouns()))
y_text_list = list(map(lambda j : j[1],train.nouns()))
vectorizer.fit_transform(x_text_list)

In [ ]:
gsvc.fit(vectorizer.transform(x_text_list), y_text_list)

In [ ]:
print(gsvc.best_score_, gsvc.best_params_)

In [ ]:
joblib.dump(gsvc, 'model.pkl')

# 머신러닝 테스트 코드

In [ ]:
testW=['앞으로 가자','뒤로 가봐','왼쪽으로 걸어','왼쪽으로 회전','좌 회전','오른쪽으로 걸어', '오른쪽으로 회전','앞 때려','공격해','왼쪽 가즈아','오른쪽 가즈아','빨리 앞으로 가라고','앞으로가라','왼쪽으로 돌지말고 오른쪽으로 가자','오른쪽으로 가지말고 왼쪽으로 돌아라','왼쪽으로 돌아','오른쪽으로 돌아'] 

In [ ]:
test=textList()
print(test.nouns())
#for nowT in testW:
#    test.append(nowT,'0')
    
#print(test.nouns())
for now in testW:
    test.append(now,'0')
print(test.nouns())

In [ ]:
test_list = list(map(lambda i:i[0], test.nouns()))

In [ ]:
pred_list = gsvc.predict(vectorizer.transform(test_list))

In [ ]:
print(pred_list)

In [ ]:
for now,now2 in zip(pred_list,testW):
    print("예측 : ",motionDict[now],"\t 정답 : ",now2)